In [ ]:
import json
from datetime import datetime

import dotenv
from langchain.chat_models import ChatAnthropic
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.vectorstores import Chroma

from redbox.llm.llm_base import LLMHandler
from redbox.llm.prompts.core import CORE_REDBOX_PROMPT
from redbox.llm.prompts.spotlight import (
    SPOTLIGHT_KEY_ACTIONS_TASK_PROMPT,
    SPOTLIGHT_KEY_DATES_TASK_PROMPT,
    SPOTLIGHT_KEY_DISCUSSION_TASK_PROMPT,
    SPOTLIGHT_KEY_PEOPLE_TASK_PROMPT,
    SPOTLIGHT_SUMMARY_TASK_PROMPT,
)

ENV = dotenv.dotenv_values("../.env")

In [ ]:
chinese_doc = """<Doc1>习近平（1953年6月15日—），男，陕西富平人，生于北京，祖籍河南邓州[註 3]，中国共产党、中华人民共和国政治人物，正国级领导人，2012年11月至今任中共中央总书记及中共中央军委主席，2013年3月至今任中华人民共和国主席及国家中央军委主席，是中国党、政、军当前及第五代的最高领导人[8].
习近平在北京出生并长大，是中国开国元老习仲勋与其第二任夫人齐心的长子，也是首位出生在中华人民共和国成立后的中共最高领导人。习近平在北京接受了中小学教育，1969年，因文化大革命对家庭带来的冲击而被迫中止学业，作为知识青年前往陕西省延安市延川县梁家河村参加劳动与工作，在此期间于1974年1月10日加入中国共产党，并在后期担任了梁家河的村党支部书记。1975年进入清华大学化工系就读，1979年毕业后先后任国务院办公厅及中央军委办公厅秘书。1982年，离京赴河北省正定县先后任县委副书记、书记，开始在地方任职。1985年赴福建，先后在福建省厦门市、宁德地区、福州市任职，1999年任福建省人民政府省长，成为正部级官员。2002年起，先后任中共浙江省委书记和中共上海市委书记。2007年10月，当选为中共中央政治局常委和中共中央书记处书记，并先后兼任或当选中共中央党校校长、国家副主席、党和国家中央军委副主席等职务。
2012年11月，在中共十八届一中全会当选为中共中央总书记和中共中央军委主席（2017、2022年获得连任），2013年3月当选为中国国家主席和国家中央军委主席（2018、2023年全票获得连任），自從行修憲打破任期制以來，是唯一連任三屆國家主席的領導人[9][10][11]，成为党和国家最高领导人至今。2016年10月，在中共十八届六中全会上确立了其领导核心的地位[12]，兼任中国共产党第十五届中央候补委员，第十六届至第二十届中央委员，第十七届至第二十届中央政治局委员、常委，第十八至二十届中央委员会总书记。第九届至第十四届全国人大代表。
习近平第一任妻子是柯玲玲。现任妻子为中国女高音歌唱家彭丽媛，育有一女习明泽</Doc1>"""

with open(
    "../data/dev/file/Russie _ la détention de la journaliste russo-américaine Alsu Kurmasheva prolongée au moins jusqu’au 23 octobre.pdf.json"
) as f:
    french_doc = json.load(f)

example_doc_with_actions = """<Doc1> The Chancellor, Jeremy Hunt and the Prime Minister Rishi Sunak met to discuss the economic recovery from COVID-19. \
The Chancellor asked the Prime Minister to write to the Bank of England to request a review of the current interest rates. \
The Prime Minister agreed to do so and will write to the Bank of England on 2023-10-23. \
Last month the Bank of England rose interest rates by 0.25% to help control inflation. </Doc1>"""

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])

handler = LLMHandler(
    llm=llm,
    user_uuid="dev",
    vector_store=Chroma(
        embedding_function=SentenceTransformerEmbeddings(),
        persist_directory="../data/dev/db",
    ),
)

In [ ]:
test_user_info = {
    "name": "Liam Wilkinson",
    "email": "liam.wilkinson@cabinetoffice.gov.uk",
    "department": "Cabinet Office",
    "role": "Economic Policy",
    "preffered_language": "British English",
}

In [ ]:
messages = [
    SystemMessage(
        content=CORE_REDBOX_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(content="Hi!"),
]

resp = llm(messages)
print(resp.content)

In [ ]:
# UK Example

messages = [
    SystemMessage(
        content=SPOTLIGHT_KEY_PEOPLE_TASK_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(
        content="<Doc1> The Chancellor, Jeremy Hunt and the Prime Minister Rishi Sunak met to discuss the economic recovery from COVID-19.</Doc1>"
    ),
]

resp = llm(messages)

print(resp.content)

In [ ]:
# Chinese Example

messages = [
    SystemMessage(
        content=SPOTLIGHT_KEY_PEOPLE_TASK_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(content=chinese_doc),
]

resp = llm(messages=messages)

print(resp.content)

In [ ]:
messages = [
    SystemMessage(
        content=SPOTLIGHT_KEY_ACTIONS_TASK_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(content=example_doc_with_actions),
]

resp = llm(messages)

print(resp.content)

In [ ]:
messages = [
    SystemMessage(
        content=SPOTLIGHT_KEY_DATES_TASK_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(content=example_doc_with_actions),
]

resp = llm(messages)

print(resp.content)

In [ ]:
with open("../data/dev/file/Widget Policy proposal.eml.json") as f:
    file = json.load(f)


messages = [
    SystemMessage(
        content=SPOTLIGHT_KEY_DISCUSSION_TASK_PROMPT.format(
            user_info=test_user_info, current_datetime=datetime.now().isoformat()
        )
    ),
    HumanMessage(content=f"<Doc1>{file['text']}</Doc1>"),
]

resp = llm(messages)

print(resp.content)